In [ ]:
# Importamos librerias de trabajo
import pandas as pd
import numpy as np
import uuid
from nltk import * 
#https://sites.google.com/view/programacion-en-python/home/5-etiquetado-morfosint%C3%A1tico
#https://www.mygreatlearning.com/blog/pos-tagging/
#https://towardsdatascience.com/part-of-speech-tagging-with-hidden-markov-chain-models-e9fccc835c0e


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import plotly.express as px


**Estructura del wikicorpus**

El corpus se compone de frases en español etiquetadas con conocimiento sobre las partes de la oración (categorías gramaticales o POS tags). Estas frases etiquetadas han sido extraídas de algunos documentos que forman parte de Wikicorpus, un corpus trilingüe (español, catalán e inglés) compuesto por más de 750 millones de palabras. Wikicorpus fue creado por investigadores de la Universitat Politèncnica de Catalunya a partir de documentos de la Wikipedia que fueron anotados con la librería opensource FreeLing.

![image.png](attachment:image.png)

Las etiquetas gramaticales (POS tags) utilizadas para anotar la información morfosintáctica del corpus son las definidas en FreeLing y se basan en EAGLES, una recomendación para la anotación de la mayoría de las lenguas europeas. La definición del conjunto de etiquetas gramaticales (POS tags) utilizadas por FreeLing en el etiquetado de un corpus en español se puede consultar en la web.


## Spanish target set
Las etiquetas gramaticales (POS tags) utilizadas para anotar la información morfosintáctica del corpus son las definidas en FreeLing y se basan en EAGLES, una recomendación para la anotación de la mayoría de las lenguas europeas. La definición del conjunto de etiquetas gramaticales (POS tags) utilizadas por FreeLing en el etiquetado de un corpus en español se puede consultar en la web.

Freeling: 

https://freeling-user-manual.readthedocs.io/en/v4.1/tagsets/tagset-es/

Para estructurar los datos y poderlos leer en python se realizó el siguiente procedimiento:
1. Se extrajo el codigo fuente de la pagina de Freeling y el codigo HTML resultante de cada tabla se parsearon con las herramientas descritas debajo
1. [Herramienta de conversion HTML a JSON](http://beautifytools.com/html-to-json-converter.php)
1. [Herramienta para compactar JSON](https://jsonformatter.org/json-minify)

In [ ]:
#Funcion para estructurar los datos del etiquetado gramatical

def create_postag_dataframe():
    targetset = list()
    adjetive = [{"position":"0","atribute":"category","values":"A:adjective"},{"position":"1","atribute":"type","values":"O:ordinal; Q:qualificative; P:possessive"},{"position":"2","atribute":"degree","values":"S:superlative; V:evaluative"},{"position":"3","atribute":"gen","values":"F:feminine; M:masculine; C:common"},{"position":"4","atribute":"num","values":"S:singular; P:plural; N:invariable"},{"position":"5","atribute":"possessorpers","values":"1:1; 2:2; 3:3"},{"position":"6","atribute":"possessornum","values":"S:singular; P:plural; N:invariable"}]
    conjuntion=[{"position":"0","atribute":"category","values":"C:conjunction"},{"position":"1","atribute":"type","values":"C:coordinating; S:subordinating"}]
    determiner=[{"position":"0","atribute":"category","values":"D:determiner"},{"position":"1","atribute":"type","values":"A:article; D:demonstrative; I:indefinite; P:possessive; T:interrogative; E:exclamative"},{"position":"2","atribute":"person","values":"1:1; 2:2; 3:3"},{"position":"3","atribute":"gen","values":"F:feminine; M:masculine; C:common"},{"position":"4","atribute":"num","values":"S:singular; P:plural; N:invariable"},{"position":"5","atribute":"possessornum","values":"S:singular; P:plural; N:invariable"}]
    noun=[{"position":"0","atribute":"category","values":"N:noun"},{"position":"1","atribute":"type","values":"C:common; P:proper"},{"position":"2","atribute":"gen","values":"F:feminine; M:masculine; C:common"},{"position":"3","atribute":"num","values":"S:singular; P:plural; N:invariable"},{"position":"4","atribute":"neclass","values":"S:person; G:location; O:organization; V:other"},{"position":"5","atribute":"nesubclass","values":"Not used"},{"position":"6","atribute":"degree","values":"V:evaluative"}]
    pronoun=[{"position":"0","atribute":"category","values":"P:pronoun"},{"position":"1","atribute":"type","values":"D:demonstrative;   E:exclamative;   I:indefinite;   P:personal;   R:relative;   T:interrogative"},{"position":"2","atribute":"person","values":"1:1;   2:2;   3:3"},{"position":"3","atribute":"gen","values":"F:feminine;   M:masculine;   C:common"},{"position":"4","atribute":"num","values":"S:singular;   P:plural;   N:invariable"},{"position":"5","atribute":"case","values":"N:nominative;   A:accusative;   D:dative;   O:oblique"},{"position":"6","atribute":"polite","values":"P:yes"}]
    adverb=[{"position":"0","atribute":"category","values":"R:adverb"},{"position":"1","atribute":"type","values":"N:negative;   G:general"}]
    adposition=[{"position":"0","atribute":"category","values":"S:adposition"},{"position":"1","atribute":"type","values":"P:preposition"}]
    verb=[{"position":"0","atribute":"category","values":"V:verb"},{"position":"1","atribute":"type","values":"M:main;   A:auxiliary;   S:semiauxiliary"},{"position":"2","atribute":"mood","values":"I:indicative;   S:subjunctive;   M:imperative;   P:participle;   G:gerund;   N:infinitive"},{"position":"3","atribute":"tense","values":"P:present;   I:imperfect;   F:future;   S:past;   C:conditional"},{"position":"4","atribute":"person","values":"1:1;   2:2;   3:3"},{"position":"5","atribute":"num","values":"S:singular;   P:plural"},{"position":"6","atribute":"gen","values":"F:feminine;   M:masculine;   C:common"}]
    number=[{"position":"0","atribute":"category","values":"Z:number"},{"position":"1","atribute":"type","values":"d:partitive;   m:currency;   p:percentage;   u:unit"}]
    date=[{"position":"0","atribute":"category","values":"W:date"}]
    interjection=[{"position":"0","atribute":"category","values":"I:interjection"}]
    punctuation=[{"tag":"Fd","attributes":"pos:punctuation;   type:colon"},{"tag":"Fc","attributes":"pos:punctuation;   type:comma"},{"tag":"Flt","attributes":"pos:punctuation;  type:curlybracket;   punctenclose:close"},{"tag":"Fla","attributes":"pos:punctuation;  type:curlybracket;   punctenclose:open"},{"tag":"Fs","attributes":"pos:punctuation;   type:etc"},{"tag":"Fat","attributes":"pos:punctuation;  type:exclamationmark;   punctenclose:close"},{"tag":"Faa","attributes":"pos:punctuation;  type:exclamationmark;   punctenclose:open"},{"tag":"Fg","attributes":"pos:punctuation;   type:hyphen"},{"tag":"Fz","attributes":"pos:punctuation;   type:other"},{"tag":"Fpt","attributes":"pos:punctuation;  type:parenthesis;   punctenclose:close"},{"tag":"Fpa","attributes":"pos:punctuation;  type:parenthesis;   punctenclose:open"},{"tag":"Ft","attributes":"pos:punctuation;   type:percentage"},{"tag":"Fp","attributes":"pos:punctuation;   type:period"},{"tag":"Fit","attributes":"pos:punctuation;  type:questionmark;   punctenclose:close"},{"tag":"Fia","attributes":"pos:punctuation;  type:questionmark;   punctenclose:open"},{"tag":"Fe","attributes":"pos:punctuation;   type:quotation"},{"tag":"Frc","attributes":"pos:punctuation;  type:quotation;   punctenclose:close"},{"tag":"Fra","attributes":"pos:punctuation;  type:quotation;   punctenclose:open"},{"tag":"Fx","attributes":"pos:punctuation;   type:semicolon"},{"tag":"Fh","attributes":"pos:punctuation;   type:slash"},{"tag":"Fct","attributes":"pos:punctuation;  type:squarebracket;   punctenclose:close"},{"tag":"Fca","attributes":"pos:punctuation;  type:squarebracket;   punctenclose:open"}]

    targetset.append(adjetive)
    targetset.append(conjuntion)
    targetset.append(determiner)
    targetset.append(noun)
    targetset.append(pronoun)
    targetset.append(adverb)
    targetset.append(adposition)
    targetset.append(verb)
    targetset.append(number)
    targetset.append(date)
    targetset.append(interjection)
    targetset.append(punctuation)

    # iteración para agregar nuevas columnas(group/category) para agrupar los registros
    for lista in targetset:
        categoria_dict=lista[0]
        grupo=""
        categoria=""

        if("values" in categoria_dict):
            grupo = categoria_dict["values"]
            categoria = grupo.split(":")[0]
        else:
            grupo = "F:punctuation"
            categoria = "F"

        for dictionario in lista:
            dictionario["group"] = grupo
            dictionario["category"] = categoria
            
    #dataframe que almacenará todos los registros agrupados
    df = pd.DataFrame()

    for lista in targetset:
        df = df.append(pd.DataFrame(lista))

    df = df.reset_index()
    return df

#Crear base de datos del etiquetado gramatical
postag_df = create_postag_dataframe()


postag_df.head(10)

,index,position,atribute,values,group,category,tag,attributes
0,0,0,category,A:adjective,A:adjective,A,NaN,NaN
1,1,1,type,O:ordinal; Q:qualificative; P:possessive,A:adjective,A,NaN,NaN
2,2,2,degree,S:superlative; V:evaluative,A:adjective,A,NaN,NaN
3,3,3,gen,F:feminine; M:masculine; C:common,A:adjective,A,NaN,NaN
4,4,4,num,S:singular; P:plural; N:invariable,A:adjective,A,NaN,NaN
5,5,5,possessorpers,1:1; 2:2; 3:3,A:adjective,A,NaN,NaN
6,6,6,possessornum,S:singular; P:plural; N:invariable,A:adjective,A,NaN,NaN
7,0,0,category,C:conjunction,C:conjunction,C,NaN,NaN
8,1,1,type,C:coordinating; S:subordinating,C:conjunction,C,NaN,NaN
9,0,0,category,D:determiner,D:determiner,D,NaN,NaN


In [ ]:
# Funcion para parsear WikiCorpus

def parse_corpus(file_name):
    column_names = ["token", "lema", "pos_tag","categoria","wordtag","bigrama","sentido","tipo","modo","tiempo","persona","numero","genero"]
    data = pd.DataFrame(columns = column_names)
    #Leer archivo a procesar. Nota : 'utf-8' para leer acentos
    textFile = open(file_name, "r",encoding='utf-8')
    index=0
    
    #Leer lineas del archivo
    for line in textFile:
        #line = line.replace("\n", "")
        splits = line.split(" ")
        
        if(len(splits)==1):continue
        if(splits[0]=="<doc"):continue
            
        #['Luis_Buñuel', 'luis_buñuel', 'NP00000', '0\n']
        eagle    = splits[2]
        categoria= eagle[0:2]
            
            
        frase = {'token' : splits[0] , 
                 'lema' : splits[1],
                 'pos_tag':eagle,
                 'categoria':categoria,
                 'wordtag': splits[0].lower()+'|'+categoria,
                 'sentido':splits[3].replace("\n", ""),
                 'tipo':eagle[1:2],
                 'modo':eagle[2:3],
                 'tiempo':eagle[3:4],
                 'persona':eagle[4:5],
                 'numero':eagle[5:6],
                 'genero':eagle[6:7],
                } 
        data = data.append(frase, ignore_index=True)      

    textFile.close()
    return data

# Extraer corpus y parsearlo
corpus_df = parse_corpus("mia07_t3_ag_Corpus-tagged.txt")

def rowIndex(row):
    rowIndex = int(row.name)
    #print("{} {}".format(rowIndex,corpus_df.shape[0]))
    
    return int(rowIndex+1)

corpus_df['bigrama'] = [corpus_df.iloc[i]['categoria']+'|'+corpus_df.iloc[i-1]['categoria'] if i > 0 else corpus_df.iloc[i]['categoria']+'|' for i in range(corpus_df.shape[0])]



In [ ]:
corpus_df.head()

,token,lema,pos_tag,categoria,wordtag,bigrama,sentido,tipo,modo,tiempo,persona,numero,genero
0,Tristana,tristana,NP00000,NP,tristana|NP,NP|,0,P,0,0,0,0,0
1,es,ser,VSIP3S0,VS,es|VS,VS|NP,01775973,S,I,P,3,S,0
2,una,uno,DI0FS0,DI,una|DI,DI|VS,0,I,0,F,S,0,
3,película,película,NCFS000,NC,película|NC,NC|DI,04960631,C,F,S,0,0,0
4,de,de,SPS00,SP,de|SP,SP|NC,0,P,S,0,0,,


In [ ]:
tagCountDict = dict()
emissionDict = dict()
transitionDict = dict()

tagCountDict = dict(zip(corpus_df['categoria'].value_counts().index, corpus_df['categoria'].value_counts().values))
emissionDict = dict(zip(corpus_df['wordtag'].value_counts().index, corpus_df['wordtag'].value_counts().values))
transitionDict = dict(zip(corpus_df['bigrama'].value_counts().index, corpus_df['bigrama'].value_counts().values))

In [ ]:
transitionDict

{'AO|DP': 1,
 'AQ|CC': 2,
 'AQ|DA': 3,
 'AQ|DI': 3,
 'AQ|Fe': 1,
 'AQ|NC': 9,
 'AQ|RG': 3,
 'AQ|RN': 1,
 'AQ|VM': 3,
 'AQ|VS': 2,
 'CC|AQ': 4,
 'CC|Fc': 4,
 'CC|Fp': 1,
 'CC|NC': 6,
 'CC|NP': 2,
 'CC|RG': 2,
 'CC|VM': 4,
 'CS|AQ': 2,
 'CS|CC': 1,
 'CS|DD': 1,
 'CS|Fp': 2,
 'CS|RG': 1,
 'CS|SP': 4,
 'CS|VM': 6,
 'DA|CS': 2,
 'DA|Fc': 2,
 'DA|Fe': 1,
 'DA|NC': 1,
 'DA|SP': 27,
 'DA|VM': 7,
 'DD|NC': 1,
 'DD|SP': 3,
 'DI|CS': 2,
 'DI|DA': 1,
 'DI|Fc': 2,
 'DI|RG': 1,
 'DI|SP': 7,
 'DI|VA': 1,
 'DI|VM': 6,
 'DI|VS': 1,
 'DP|CS': 1,
 'DP|SP': 3,
 'Fc|AQ': 2,
 'Fc|Fp': 1,
 'Fc|NC': 7,
 'Fc|NP': 6,
 'Fc|RG': 1,
 'Fc|VM': 2,
 'Fe|AQ': 1,
 'Fe|NC': 1,
 'Fe|NP': 1,
 'Fe|P0': 1,
 'Fe|VM': 2,
 'Fp|Fe': 2,
 'Fp|NC': 6,
 'Fp|NP': 9,
 'Fp|RG': 1,
 'Fp|VM': 3,
 'Fp|Z': 1,
 'Fx|AQ': 1,
 'NC|AO': 1,
 'NC|AQ': 7,
 'NC|CC': 2,
 'NC|CS': 1,
 'NC|DA': 28,
 'NC|DD': 2,
 'NC|DI': 16,
 'NC|DP': 3,
 'NC|SP': 7,
 'NC|VM': 3,
 'NC|VS': 1,
 'NC|Z': 2,
 'NP|': 1,
 'NP|AQ': 1,
 'NP|CC': 1,
 'NP|CS': 2,
 'NP|DA': 1,


In [ ]:
transitionProbDict = {} # matriz A
emissionProbDict = {} # Matriz B

# Probabilidades de transición
# Se crea un for que recorre todos las llaves del diccionario de transición
for key in transitionDict.keys():

  # Se separan los elementos
  tag, prevtag = key.split('|')
  if prevtag is not '':
    # Si la etiqueta aparece individual en el diccionario
    if tagCountDict[prevtag]>0:
      # Entonces se calcula la probabilidad
      transitionProbDict[key] = transitionDict[key]/(tagCountDict[prevtag])
    
    else:
      print(key)

#Probabilidades de Emisión
for key in emissionDict.keys():
  word , tag = key.split('|')

  # Si en el diccionario de emisión la pareja como tal tiene un conteo mayor que 0
  if emissionDict[key]>0:
    # Calcular las probabilidades de la etiqueta de la palabra
    emissionProbDict[key] = emissionDict[key]/tagCountDict[tag]
  else:
    print(key)

In [ ]:
transitionProbDict

{'AO|DP': 0.25,
 'AQ|CC': 0.08695652173913043,
 'AQ|DA': 0.075,
 'AQ|DI': 0.14285714285714285,
 'AQ|Fe': 0.16666666666666666,
 'AQ|NC': 0.1232876712328767,
 'AQ|RG': 0.1875,
 'AQ|RN': 0.25,
 'AQ|VM': 0.04411764705882353,
 'AQ|VS': 0.25,
 'CC|AQ': 0.14814814814814814,
 'CC|Fc': 0.21052631578947367,
 'CC|Fp': 0.045454545454545456,
 'CC|NC': 0.0821917808219178,
 'CC|NP': 0.0625,
 'CC|RG': 0.125,
 'CC|VM': 0.058823529411764705,
 'CS|AQ': 0.07407407407407407,
 'CS|CC': 0.043478260869565216,
 'CS|DD': 0.25,
 'CS|Fp': 0.09090909090909091,
 'CS|RG': 0.0625,
 'CS|SP': 0.058823529411764705,
 'CS|VM': 0.08823529411764706,
 'DA|CS': 0.11764705882352941,
 'DA|Fc': 0.10526315789473684,
 'DA|Fe': 0.16666666666666666,
 'DA|NC': 0.0136986301369863,
 'DA|SP': 0.39705882352941174,
 'DA|VM': 0.10294117647058823,
 'DD|NC': 0.0136986301369863,
 'DD|SP': 0.04411764705882353,
 'DI|CS': 0.11764705882352941,
 'DI|DA': 0.025,
 'DI|Fc': 0.10526315789473684,
 'DI|RG': 0.0625,
 'DI|SP': 0.10294117647058823,
 'DI|VA

In [ ]:
# identificamos las categorias gramaticales 'upos' unicas en el corpus
stateSet = set([w.split('|')[1] for w in list(emissionProbDict.keys())])
stateSet

{'AO',
 'AQ',
 'CC',
 'CS',
 'DA',
 'DD',
 'DI',
 'DP',
 'Fc',
 'Fe',
 'Fp',
 'Fx',
 'NC',
 'NP',
 'P0',
 'PI',
 'PP',
 'PR',
 'RG',
 'RN',
 'SP',
 'VA',
 'VM',
 'VS',
 'Z',
 'Zu'}

In [ ]:
# enumeramos las categorias con numeros para asignar a 
# las columnas de la matriz de Viterbi
tagStateDict = {}
for i, state in enumerate(stateSet):
  tagStateDict[state] = i
tagStateDict

{'AO': 16,
 'AQ': 12,
 'CC': 7,
 'CS': 11,
 'DA': 20,
 'DD': 13,
 'DI': 3,
 'DP': 8,
 'Fc': 10,
 'Fe': 22,
 'Fp': 19,
 'Fx': 17,
 'NC': 4,
 'NP': 25,
 'P0': 15,
 'PI': 5,
 'PP': 14,
 'PR': 18,
 'RG': 21,
 'RN': 1,
 'SP': 9,
 'VA': 23,
 'VM': 2,
 'VS': 6,
 'Z': 0,
 'Zu': 24}

In [ ]:
# Calculamos distribución inicial de estados
initTagStateProb = {} # \rho_i^{(0)}
#from conllu import parse_incr 
wordList = []
#data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8")
count = 0 # cuenta la longitud del corpus
for index, row in corpus_df.iterrows():
  count += 1
  tag = row['categoria']
  if tag in initTagStateProb.keys():
    initTagStateProb[tag] += 1
  else:
    initTagStateProb[tag] = 1

for key in initTagStateProb.keys():
  initTagStateProb[key] /= count

initTagStateProb

{'AO': 0.0020325203252032522,
 'AQ': 0.054878048780487805,
 'CC': 0.046747967479674794,
 'CS': 0.034552845528455285,
 'DA': 0.08130081300813008,
 'DD': 0.008130081300813009,
 'DI': 0.042682926829268296,
 'DP': 0.008130081300813009,
 'Fc': 0.03861788617886179,
 'Fe': 0.012195121951219513,
 'Fp': 0.044715447154471545,
 'Fx': 0.0020325203252032522,
 'NC': 0.1483739837398374,
 'NP': 0.06504065040650407,
 'P0': 0.012195121951219513,
 'PI': 0.0040650406504065045,
 'PP': 0.026422764227642278,
 'PR': 0.018292682926829267,
 'RG': 0.032520325203252036,
 'RN': 0.008130081300813009,
 'SP': 0.13821138211382114,
 'VA': 0.006097560975609756,
 'VM': 0.13821138211382114,
 'VS': 0.016260162601626018,
 'Z': 0.008130081300813009,
 'Zu': 0.0020325203252032522}

In [ ]:
sum(initTagStateProb.values())

1.0

In [ ]:
emissionProbDict

{'"|Fe': 1.0,
 '(|Fp': 0.13636363636363635,
 ')|Fp': 0.13636363636363635,
 ',|Fc': 1.0,
 '.|Fp': 0.7272727272727273,
 '1970|Z': 0.25,
 '2|Z': 0.25,
 ';|Fx': 1.0,
 'a_la_vez|RG': 0.125,
 'a_pesar_de|SP': 0.014705882352941176,
 'accidente|NC': 0.0136986301369863,
 'actúa|VM': 0.014705882352941176,
 'acusado|VM': 0.014705882352941176,
 'adora|VM': 0.014705882352941176,
 'afortunadamente|RG': 0.0625,
 'agresivo|AQ': 0.037037037037037035,
 'amistad|NC': 0.0136986301369863,
 'antes|RG': 0.0625,
 'antimonio|NC': 0.0136986301369863,
 'apodado|VM': 0.014705882352941176,
 'apresado|VM': 0.014705882352941176,
 'así|RG': 0.0625,
 'ataque|NC': 0.0136986301369863,
 'a|SP': 0.20588235294117646,
 'aún|RG': 0.0625,
 'banalidad|NC': 0.0136986301369863,
 'basada|VM': 0.014705882352941176,
 'benito_pérez_galdós|NP': 0.03125,
 'biggs|NP': 0.03125,
 'bob_egle|NP': 0.03125,
 'bovingdon_bug|NP': 0.03125,
 'brad_pitt|NP': 0.03125,
 'burke|NP': 0.03125,
 'bypass|NC': 0.0136986301369863,
 'cables|NC': 0.01369863

In [ ]:
def ViterbiMatrix(secuencia, transitionProbdict=transitionProbDict, emissionProbdict=emissionProbDict, 
            tagStateDict=tagStateDict, initTagStateProb=initTagStateProb):
  seq = secuencia.split(' ')
  viterbiProb = np.zeros((len(tagStateDict), len(seq)))  # eagle

  # inicialización primera columna
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key]
    word_tag = seq[0].lower()+'|'+key
    if word_tag in emissionProbDict.keys():
      viterbiProb[tag_row, 0] = initTagStateProb[key]*emissionProbDict[word_tag]

  # computo de las siguientes columnas
  for col in range(1, len(seq)):
    for key in tagStateDict.keys():
      tag_row = tagStateDict[key]
      word_tag = seq[col].lower()+'|'+key
      if word_tag in emissionProbDict.keys():
        # miramos estados de la col anterior
        possible_probs = []
        for key2 in tagStateDict.keys(): 
          tag_row2 = tagStateDict[key2]
          tag_prevtag = key+'|'+key2
          if tag_prevtag in transitionProbDict.keys():
            for i in range(0, len(viterbiProb)):
              if viterbiProb[i, col-1]>0:
                possible_probs.append(viterbiProb[i,col-1]*
                                      transitionProbDict[tag_prevtag]*
                                      emissionProbDict[word_tag])
                
                viterbiProb[tag_row, col] = max(possible_probs) #if possible_probs else 0
  
  return viterbiProb

matrix = ViterbiMatrix('Habla con el enfermo grave de trasplantes')
matrix

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.13008130e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.03252033e-03, 0.00000000e+00, 0.00000000e+00, 6.09662164e-06,
        0.00000000e+00, 0.00000000e+00, 1.70578768e-09],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0

In [ ]:
def ViterbiTags(secuencia, transitionProbdict=transitionProbDict, emissionProbdict=emissionProbDict, 
            tagStateDict=tagStateDict, initTagStateProb=initTagStateProb):
  seq = secuencia.split(' ')
  viterbiProb = np.zeros((len(tagStateDict), len(seq)))  # upos tiene 17 categorias

  # inicialización primera columna
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key]
    word_tag = seq[0].lower()+'|'+key
    if word_tag in emissionProbDict.keys():
      viterbiProb[tag_row, 0] = initTagStateProb[key]*emissionProbDict[word_tag]

  # computo de las siguientes columnas
  for col in range(1, len(seq)):
    for key in tagStateDict.keys():
      tag_row = tagStateDict[key]
      word_tag = seq[col].lower()+'|'+key
      if word_tag in emissionProbDict.keys():
        # miramos estados de la col anterior
        possible_probs = []
        for key2 in tagStateDict.keys(): 
          tag_row2 = tagStateDict[key2]
          tag_prevtag = key+'|'+key2
          if tag_prevtag in transitionProbDict.keys():
            if viterbiProb[tag_row2, col-1]>0:
              possible_probs.append(
                  viterbiProb[tag_row2, col-1]*transitionProbDict[tag_prevtag]*emissionProbDict[word_tag])
              viterbiProb[tag_row, col] = max(possible_probs)

    # contruccion de secuencia de tags
    res = []
    for i, p in enumerate(seq):
      for tag in tagStateDict.keys():
        if tagStateDict[tag] == np.argmax(viterbiProb[:, i]):
          res.append((p, tag))
      
  return res

ViterbiTags('Habla con el enfermo grave de trasplantes')

[('Habla', 'VM'),
 ('con', 'SP'),
 ('el', 'DA'),
 ('enfermo', 'NC'),
 ('grave', 'AQ'),
 ('de', 'SP'),
 ('trasplantes', 'NC')]